<a href="https://colab.research.google.com/github/usamathakur87/class-projects/blob/main/rag_by_usama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [4]:
import time

index_name = "online-rag-project-2"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] =  userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [6]:
vector = embeddings.embed_query("we are building a RAG Text")

In [7]:
vector[:5]

[0.04060795158147812,
 -0.04385491460561752,
 -0.05683692544698715,
 -0.021194666624069214,
 0.010449625551700592]

In [8]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [9]:
# data save

from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [10]:
len(documents)

10

In [11]:
from uuid import uuid4
uuid4()

UUID('4c21eff3-cacc-463a-b066-546d872b3d02')

In [12]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)



['169d7602-15af-42da-8a06-5887e8d89341',
 '323d4acc-10ff-4af8-8479-3611384a864e',
 '96f84d7a-ec05-4221-bc1b-fc924e9ee6aa',
 '0c64a588-7094-4322-a448-9fccb18f94b5',
 '8a101e74-bf5e-4d3d-b308-92dae2dd2829',
 '442f7ef7-3e72-47f4-a389-4e555bd9da5e',
 '49ac4a0e-68fb-4b92-9340-56c497ef1fbc',
 '0e716964-ef42-4685-a480-1f3f5df81d11',
 '605a7290-1003-4329-9ee5-1d06916c4014',
 'deb2ca34-12c8-4431-ab54-ea08280d0b7a']

In [13]:
# Data Retrive

results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [14]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [16]:
def answer_to_user(query: str):

    ## Vector Search
    # results = vector_store.similarity_search(query, k=2)
    vector_results = vector_store.similarity_search(query, k=2)
    print(len(vector_results))

    ## Todo - Pass To Model Vector Results + User Query
    # final_answer = model(query, results)
    final_answer = llm.invoke(f"ANSWER THIS USER QUERY: {query}, Here are some references to asnwer {vector_results} ")

    return final_answer


In [17]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")


2


In [18]:
answer.content


'Based on the provided documents, here\'s what we can say about LangChain:\n\n*   **LangChain is a framework for building applications.** The second tweet explicitly mentions "Building an exciting new project with LangChain".\n*   **LangChain is used for building agentic applications.** The first tweet highlights "LangGraph" as the best framework for building "stateful, agentic applications". While it mentions LangGraph, it implies that LangChain is related to building agentic applications.\n*   **LangChain is considered exciting.** The second tweet uses the phrase "exciting new project" when referring to using LangChain.\n\n**In summary, based on these limited references, LangChain is a framework used for building applications, particularly agentic ones, and is considered an exciting tool to work with.**\n\nIt\'s important to note that these are very limited references. We don\'t have any information about the specific abstractions LangChain provides or how it makes working with LLMs 